# NVTabular / HugeCTR Criteo Example 
Here we'll show how to use NVTabular first as a preprocessing library to prepare the [Criteo Display Advertising Challenge](https://www.kaggle.com/c/criteo-display-ad-challenge) dataset, and then train a model using HugeCTR.

### Data Prep
Before we get started, make sure you've run the [`optimize_criteo` notebook](./optimize_criteo.ipynb), which will convert the tsv data published by Criteo into the parquet format that our accelerated readers prefer. It's fair to mention at this point that that notebook will take ~4 hours to run. While we're hoping to release accelerated csv readers in the near future, we also believe that inefficiencies in existing data representations like csv are in no small part a consequence of inefficiencies in the existing hardware/software stack. Accelerating these pipelines on new hardware like GPUs may require us to make new choices about the representations we use to store that data, and parquet represents a strong alternative.

#### Quick Aside: Clearing Cache
The following line is not strictly necessary, but is included for those who want to validate NVIDIA's benchmarks. We start by clearing the existing cache to start as "fresh" as possible. If you're having trouble running it, try executing the container with the `--privileged` flag.

In [1]:
!sync; echo 3 > /proc/sys/vm/drop_caches

/bin/sh: 1: cannot create /proc/sys/vm/drop_caches: Read-only file system


In [2]:
import os
from time import time
import re
import glob
import warnings

# tools for data preproc/loading
import torch
import rmm
import nvtabular as nvt
from nvtabular.ops import Normalize, FillMissing, Categorify, Moments, Median, LogOp, ZeroFill, get_embedding_sizes
from nvtabular.torch_dataloader import DLDataLoader, TorchTensorBatchDatasetItr, create_tensors_plain

/conda/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/conda/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice/.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


### Initializing the Memory Pool
For applications like the one that follows where RAPIDS will be the only workhorse user of GPU memory and resource, a good best practices is to use the RAPIDS Memory Manager library `rmm` to allocate a dedicated pool of GPU memory that allows for fast, asynchronous memory management. Here, we'll dedicate 80% of free GPU memory to this pool to make sure we get the most utilization possible.

In [3]:
rmm.reinitialize(pool_allocator=True, initial_pool_size=0.8 * nvt.io.device_mem_size(kind='free'))

/nvtabular/nvtabular/io.py:86: UserWarning: get_memory_info is not supported. Using total device memory from NVML.
  warnings.warn("get_memory_info is not supported. Using total device memory from NVML.")


### Dataset and Dataset Schema
Once our data is ready, we'll define some high level parameters to describe where our data is and what it "looks like" at a high level.

In [4]:
# define some information about where to get our data
INPUT_DATA_DIR = os.environ.get('INPUT_DATA_DIR', '/dataset')
OUTPUT_DATA_DIR = os.environ.get('OUTPUT_DATA_DIR', '/dataset/output') # where we'll save our procesed data to
!mkdir $OUTPUT_DATA_DIR
NUM_TRAIN_DAYS = 23 # number of days worth of data to use for training, the rest will be used for validation

# define our dataset schema
CONTINUOUS_COLUMNS = ['I' + str(x) for x in range(1,14)]
CATEGORICAL_COLUMNS =  ['C' + str(x) for x in range(1,27)]
LABEL_COLUMNS = ['label']
COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS + LABEL_COLUMNS

mkdir: cannot create directory ‘/dataset/output’: File exists


In [5]:
# ! ls $INPUT_DATA_DIR

In [6]:
fname = 'day_{}.parquet'
num_days = len([i for i in os.listdir(INPUT_DATA_DIR) if re.match(fname.format('[0-9]{1,2}'), i) is not None])
train_paths = [os.path.join(INPUT_DATA_DIR, fname.format(day)) for day in range(1)]
valid_paths = [os.path.join(INPUT_DATA_DIR, fname.format(day)) for day in range(NUM_TRAIN_DAYS, NUM_TRAIN_DAYS+1)]
print(train_paths)
print(valid_paths)

['/dataset/day_0.parquet']
['/dataset/day_23.parquet']


### Preprocessing
At this point, our data still isn't in a form that's ideal for consumption by neural networks. The most pressing issues are missing values and the fact that our categorical variables are still represented by random, discrete identifiers, and need to be transformed into contiguous indices that can be leveraged by a learned embedding. Less pressing, but still important for learning dynamics, are the distributions of our continuous variables, which are distributed across multiple orders of magnitude and are uncentered (i.e. E[x] != 0).

We can fix these issues in a conscise and GPU-accelerated manner with an NVTabular `Workflow`. We'll instantiate one with our current dataset schema, then symbolically add operations _on_ that schema. By setting all these `Ops` to use `replace=True`, the schema itself will remain unmodified, while the variables represented by each field in the schema will be transformed.

#### Frequency Thresholding
One interesting thing worth pointing out is that we're using _frequency thresholding_ in our `Categorify` op. This handy functionality will map all categories which occur in the dataset with some threshold level of infrequency (which we've set here to be 15 occurrences throughout the dataset) to the _same_ index, keeping the model from overfitting to sparse signals.

In [7]:
proc = nvt.Workflow(
    cat_names=CATEGORICAL_COLUMNS,
    cont_names=CONTINUOUS_COLUMNS,
    label_name=LABEL_COLUMNS)

# log -> normalize continuous features. Note that doing this in the opposite
# order wouldn't make sense! Note also that we're zero filling continuous
# values before the log: this is a good time to remember that LogOp
# performs log(1+x), not log(x)
proc.add_cont_feature([ZeroFill(), LogOp()])
proc.add_cont_preprocess(Normalize())

# categorification with frequency thresholding
proc.add_cat_preprocess(Categorify(freq_threshold=15))

Now instantiate dataset iterators to loop through our dataset (which we couldn't fit into GPU memory)

In [8]:
train_dataset = nvt.Dataset(train_paths, engine='parquet', part_mem_fraction=0.12)
valid_dataset = nvt.Dataset(valid_paths, engine='parquet', part_mem_fraction=0.12)

Now run them through our workflows to collect statistics on the train set, then transform and save to parquet files.

In [9]:
output_train_dir = os.path.join(OUTPUT_DATA_DIR, 'train/')
output_valid_dir = os.path.join(OUTPUT_DATA_DIR, 'valid/')
! mkdir -p $output_train_dir
! mkdir -p $output_valid_dir

For reference, let's time it to see how long it takes...

In [13]:
%%time
proc.apply(train_dataset, apply_offline=True, record_stats=True, shuffle=False, output_format="parquet", output_path=output_train_dir, out_files_per_proc=2)

CPU times: user 1min 8s, sys: 33.2 s, total: 1min 41s
Wall time: 2min 12s


In [14]:
%%time
proc.apply(valid_dataset, apply_offline=True, record_stats=False, shuffle=False, output_format="parquet", output_path=output_valid_dir, out_files_per_proc=2)

CPU times: user 20.1 s, sys: 27 s, total: 47.1 s
Wall time: 1min 16s


And just like that, we have training and validation sets ready to feed to a model!

## HugeCTR
### Training
We'll run huge_ctr using the configuration file.

First, we'll reinitialize our memory pool from earlier to free up some memory so that we can share it with PyTorch.

In [10]:
rmm.reinitialize(pool_allocator=False)

In [13]:
! /usr/local/hugectr/bin/huge_ctr --train dcn_parquet.json

[0.001, init_start, ]
HugeCTR Version: 2.1
Config file: dcn_parquet.json
[04d18h48m11s][HUGECTR][INFO]: Default evaluation metric is AUC without threshold value
[04d18h48m13s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-DGXS-16GB
[04d18h48m13s][HUGECTR][INFO]: Initial seed is 1944391260
[04d18h48m13s][HUGECTR][INFO]: cache_eval_data is not specified using default: 0
[04d18h48m14s][HUGECTR][INFO]: Vocabulary size: 2116453
worker_id: 0 mem: 0x7f74a01e7b40
[04d18h48m14s][HUGECTR][INFO]: num_internal_buffers 1
worker_id: 0 mem: 0x7f74a01e7b40
[04d18h48m14s][HUGECTR][INFO]: num_internal_buffers 1
[04d18h48m14s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=2700000
[04d18h48m14s][HUGECTR][INFO]: All2All Warmup Start
[04d18h48m14s][HUGECTR][INFO]: All2All Warmup End
[04d18h48m16s][HUGECTR][INFO]: gpu0 start to init embedding
[04d18h48m16s][HUGECTR][INFO]: gpu0 init embedding done
[04d18h48m16s][HUGECTR][INFO]: warmup_steps is not specified using default: 1

KeyboardInterrupt: 